# 神經網路的基本原理

「神經網路」的原理並不如想像中那麼高不可攀。本來只要會用 TenserFlow 我就已經很滿意了────看 YouTube 學不難。但是在 YouTube 上瞄到 IBM 的影片["Introduction to Neural Networks and DeepLearning using stand-alone Python"](https://www.youtube.com/watch?v=oJoPbwmVBlk) 好奇於光用 python 就能做出「神經網路」真的嗎？那好哇！因為有 demo 程式就一定能動手去 trace 也就是一定能多少懂一點，於是就點了進去看，結果是
### 老天！印度人講數學原理 -__-
幸好這數學我聽懂了前半段，接著還是真的有短、短、短，很短、很短的 python demo 程式把他講的數學公式實現出來，最後一跑，誤差真的有收斂，我終於歡欣地確認原本想像中嚇死人的大學問，其實障礙很低。這下子我深深地著了迷，不可自已地又翻了好多 YouTube 前後參考把後半段也搞到半懂。以下就是從這兩週的失魂落魄中回轉攜來的心得報告。



## DNN「神經網路」就是 Decoder，它的手段就是「查表」

大致看懂這些數學推導之後，我心裡浮出這個結論。開始之前，我們先來複習一下這部分，有助於順順地理解「神經網路」是在幹嘛。

Decoder（解碼器）是 computer science 裡很基本的東西，學過了就很難真的忘記。任何問題只要是能用 decoder 解決的，那你用 decoder 就對了，它一定是最快最可靠的解決方案。順便複習一下，例如下圖就是簡單地用一個 decoder 實現的二進制加法器。Decoding 根本就是「查表」，從下圖左邊輸入的 a 與 b 兩個 binary bits 被 Adder 拿進去查「真值表」，根據表中的設計，立即查出什麼樣的 a,b 組合該輸出什麼樣的結果：sum 與 carry (進位)。簡單、粗暴、無腦而完美地產生正確的結果。我十幾二十歲時搞懂 decoder 之後三十多年以後的今天，CPU 內部的加法器應該還是 decoder 沒變吧？

In [2]:
from IPython.display import IFrame; IFrame("adder.pdf", width=600, height=400)

現在問題改成要讓電子計算機辨認一張大小為 300X300 由黑點、白點構成的圖片，看圖片上是「貓」還是「狗」？我們可以直接採用與 Adder 十分類似的 decoder 來處理，如下圖：

In [4]:
IFrame("cat-dog-classifier.pdf", width=600, height=400)

這回這個 decoder 功能上的名稱由 Adder 塗改成 Cat/Dog Classifier（分類器,做貓狗照片分類）; 輸入端由 a,b 兩個 bits 塗改成 300X300 圖片上的每一 pixcel 點，總共 9 萬條輸入線，圖中用左側輸入端一個大箭頭示意；而最後的輸出則塗改成三種可能結果各別的概率。整體還是個 decoder，實際作法還是「查表」，把 9 萬條輸入線當成「地址」到 Cat/Dog Classifier 裡查出對應的位置裡預設好的三個概率值。照樣無腦而正確地分辨任何一張 300X300 照片。

可是，這 Cat/Dog Classifier 肚子裡的這張表────整個宇宙都裝不下！這就是問題所在。對照 Intel 的 386 CPU 有 32 條地址線，所以它的記憶體最多是 $2^{32}$ 即 4 GBytes, 此後每增加 10 條地址線，容量就擴大 1024 倍，呈等比級數增長。 如今這個 docoder 要求的是一個 $2^{90000}$ 容量的表格，我不知道需要幾個宇宙才夠。

P.S. 如果你質疑，即使真的有那麼大的記憶容量，怎麼在每個位置裡填入預設好的三個概率值？這可能沒有想像中那麼難。因為在 9 萬個維度的空間裡（一般人類只能想像四度空間），有充分的維度讓「貓」聚在一堆、「狗」聚在另一堆，用「近鄰法」就可以區分，所以用 labeled 好的樣本訓練即可，不用真的一一去填滿，至少方向是這樣。然而，這只是閒聊，反正容量是不可能夠的。

## 神之 Decoder 不可得，那就尋找 Decoder 的簡化方案──科技革命之始

下圖是個「神經網路」的範例。最左邊的輸入 vector (也就是 array) 是 $X$。$X$ 經過連接網路 $W^{(1)}$ 加權之後，充分完全地連接到下一層「中間層」(也就是 hidden layer)。這裡豎著排列的三個圓圈代表三顆 neuron（神經元）────不要被它們的名稱嚇到了，其實一顆 neuron 就是把輸入全部加起來得 $z_i^{(2)}$，下圖中 $z^{(2)}$ 標示在 neuron 偏左一點之處，表示它是所有輸入之「和」。因為前面那麼多東西加起來的數值會很發散，所以離開 neuron 輸出前要用一個 function 把前面的「和」也就是 $z_i^{(2)}$ 來把它 mapping 到與前後層的 input / output 一致的範圍裡，這個 mapping function 有很多可以任選，統稱為 Activation Function，圖中標示為 $f_1$。$a^{(2)}$ 標示在 neuron 偏右一點之處，表示它是 Hidden layer 一整層經過 Activation Functions 之後的輸出。

最右邊（輸出層）豎著排列的三顆 neuron（神經元）的結構與 hidden layer 幾乎一樣，但是這一層的 Activation Function 可以另選，所以標示為 $f_2$ 與前一層的 $f_1$ 區別。最右邊的輸出結果標示為 $\hat{y}$ 因為一般習慣訓練用的標準答案是 $y$ 此處 $\hat{y}$ 表示我們的神經網路猜測的結果。


In [7]:
IFrame("nn-scatch-2.pdf", width=800, height=500)
